##Mount Google Drive

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


##Importing Packages

In [6]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Reshape, Flatten, Conv2D, Dense, Conv2DTranspose, Dropout
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np
!mkdir generated_images resized_images

###Unzip the file in a destination directory

In [ ]:
!unzip /content/drive/MyDrive/DATASETS/bobross2.zip -d bobross

In [18]:
images_path = "/content/bobross/bobross2/"

##Resizing Data to feed NN

In [19]:
import os
import cv2

reshape_size = (64,64)

for i,image in enumerate(os.listdir(images_path)):
  img_path = os.path.join(images_path, image)
  img = cv2.imread(img_path)
  img = cv2.resize(img, reshape_size)
  cv2.imwrite(f"resized_images/{i}.png", img)

##NN Parameters

In [20]:
img_width = 64
img_height = 64
channels = 3
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(lr = 0.0002)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [23]:
def build_generator():
  model = Sequential([
    Dense(256 * 8 * 8 , input_dim = latent_dim), #reshape the two 8 in case doent match
    LeakyReLU(alpha = 0.2),
    Reshape((8,8,256)), #reshape the two 8 in case doent match

    Conv2DTranspose(128, (4,4), strides = (2,2), padding = "same"),
    LeakyReLU(alpha = 0.2),
    
    Conv2DTranspose(128, (4,4), strides = (2,2), padding = "same"),
    LeakyReLU(alpha = 0.2),

    Conv2DTranspose(128, (4,4), strides = (2,2), padding = "same"),
    LeakyReLU(alpha = 0.2),

    Conv2D(3, (3,3), activation = "tanh", padding = "same") 
  ])

  model.summary()
  return model

generator = build_generator()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 16384)             1654784   
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 16384)             0         
                                                                 
 reshape_1 (Reshape)         (None, 8, 8, 256)         0         
                                                                 
 conv2d_transpose_3 (Conv2DT  (None, 16, 16, 128)      524416    
 ranspose)                                                       
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 conv2d_transpose_4 (Conv2DT  (None, 32, 32, 128)      262272    
 ranspose)                                            

In [33]:
def build_discriminator():
  model = Sequential([
    Conv2D(64, (3,3), padding = "same", input_shape = img_shape),
    LeakyReLU(alpha = 0.2),

    Conv2D(128, (3,3), padding = "same"),
    LeakyReLU(alpha = 0.2),

    Conv2D(128, (3,3), padding = "same"),
    LeakyReLU(alpha = 0.2),

    Conv2D(256, (3,3), padding = "same"),
    LeakyReLU(alpha = 0.2),

    Flatten(),
    Dropout(0.4),
    Dense(1, activation = "sigmoid")

  ])

  model.summary()
  return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 64, 64, 64)        1792      
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 64, 64, 64)        0         
                                                                 
 conv2d_11 (Conv2D)          (None, 64, 64, 128)       73856     
                                                                 
 leaky_re_lu_17 (LeakyReLU)  (None, 64, 64, 128)       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 64, 64, 128)       147584    
                                                                 
 leaky_re_lu_18 (LeakyReLU)  (None, 64, 64, 128)       0         
                                                                 
 conv2d_13 (Conv2D)          (None, 64, 64, 256)      

##Creating the GAN

In [34]:
GAN = Sequential([
  generator,
  discriminator
])

GAN.layers[1].trainable = False
GAN.summary()
GAN.compile(loss = "binary_crossentropy", optimizer = adam)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 64, 64, 3)         2707203   
                                                                 
 sequential_6 (Sequential)   (None, 1)                 1566977   
                                                                 
Total params: 4,274,180
Trainable params: 2,707,203
Non-trainable params: 1,566,977
_________________________________________________________________


##Output Images

In [35]:
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 2, 2
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001
    # print("%.8f" % save_name)

    # Rescale images 0 - 1
    # gen_imgs = 0.5 * gen_imgs + 0.5
    gen_imgs = (gen_imgs + 1) / 2.0
    # gen_imgs = gen_imgs * 255

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images/%.8f.png" % save_name)
    plt.close()

##Training GAN

In [36]:
from PIL import Image

array = []
path = "/content/resized_images/"

for image in os.listdir(path):
  image = Image.open(path + image)
  data = np.asarray(image)
  array.append(data)

X_train = np.array(array)
X_train = X_train /127.5 -1.
print(X_train.shape)

(2612, 64, 64, 3)


Remeber the batch size should be proportional to the amoubt of data

In [37]:
def train(epochs, batch_size = 32):
  bat_per_epo = int(X_train.shape[0] / batch_size)

  valid = np.ones((batch_size, 1))
  fakes = np.zeros((batch_size, 1))

  for epoch in range(epochs):
    for j in range(bat_per_epo):
      #Get random batch
      idx = np.random.randint(0,X_train.shape[0], batch_size)
      imgs = X_train[idx]

      #Generate Fake Images
      noise = np.random.normal(0,1, (batch_size, latent_dim))
      gen_imgs = generator.predict(noise)

      #train discriminator
      d_loss_real = discriminator.train_on_batch(imgs, valid)
      d_loss_fake = discriminator.train_on_batch(imgs,fakes)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

      noise = np.random.normal(0,1, (batch_size, latent_dim))

      #inverse y label
      g_loss = GAN.train_on_batch(noise,valid)

      print("******* %d %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch,j, d_loss[0], 100* d_loss[1], g_loss))

    save_imgs(epoch)

train(60000, batch_size = 24)


******* 0 50 [D loss: 0.699227, acc: 33.33%] [G loss: 0.599636]
******* 0 51 [D loss: 0.698795, acc: 35.42%] [G loss: 0.595855]


KeyboardInterrupt: ignored

##Create a GIF

In [ ]:
# Display a single image using the epoch number
# def display_image(epoch_no):
#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)